<a href="https://colab.research.google.com/github/srohit0/CORDIC/blob/master/examples/food_MNIST_keras_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# food MNIST with Keras ResNet-18


<img src=https://raw.githubusercontent.com/srohit0/food_mnist/master/images/food-collage.jpg  width="600" height="300" align="center">


food dataset is a challanging dataset for most networks. This notebook shows results of applying resnet on food-MNIST with and without augmentation. It is easy to see that accuracy increases with augmentation supporting deep learning principle of:
### more data, better results
---

## Clone dataset 

In [0]:
! wget -q https://raw.githubusercontent.com/raghakot/keras-resnet/master/resnet.py -O resnet.py
! [ ! -d "food_mnist" ] &&  git clone -q https://github.com/srohit0/food_mnist.git

## Adapt Dataset

In [0]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels



def divide_dataset(dataset, labels):
    train_pct = 0.80; 
    # divide dataset into training and validation set
    train_index = int(dataset.shape[0]*train_pct)
    t_X = dataset[:train_index, :]
    t_Y = labels[:train_index]
    v_X = dataset[train_index:,:]
    v_Y = labels[train_index:]
    
    return (t_X, t_Y), (v_X, v_Y)

In [3]:
from __future__ import print_function
import food_mnist
import resnet

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping

import numpy as np


lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger('resnet18_cifar10.csv')

batch_size = 32
nb_classes = 10
epochs = 200

# input image dimensions
img_rows, img_cols, img_channels= 32, 32, 3

# The data, shuffled and split between train and test sets:
(X_train, Y_train), (X_test, Y_test) = food_mnist.load_data(img_cols, img_rows)

X = np.concatenate((X_train, X_test), axis=0)
Y = np.concatenate((Y_train, Y_test), axis=0)

X, Y = randomize(X, Y)
(X_train, Y_train), (X_test, Y_test) = divide_dataset(X, Y)

# Convert class vectors to binary class matrices.
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# subtract mean and normalize
mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_test -= mean_image
X_train /= 128.
X_test /= 128.

Using TensorFlow backend.


## Create Deep Learning Model (ResNet-18)
---


<img src=https://www.researchgate.net/profile/Paolo_Napoletano/publication/322476121/figure/tbl1/AS:668726449946625@1536448218498/ResNet-18-Architecture_W640.jpg height="400">

In [4]:
model = resnet.ResnetBuilder.build_resnet_18((img_channels, img_rows, img_cols), nb_classes)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



W0622 00:04:03.561514 140196683085696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0622 00:04:03.582317 140196683085696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0622 00:04:03.585685 140196683085696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0622 00:04:03.613817 140196683085696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0622 00:04:03.614673

## Train

In [0]:
def Train(data_augmentation):
  if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_test, Y_test),
              shuffle=True,
              callbacks=[lr_reducer, early_stopper, csv_logger])
  else:
      print('Using real-time data augmentation.')
      # This will do preprocessing and realtime data augmentation:
      datagen = ImageDataGenerator(
          featurewise_center=False,  # set input mean to 0 over the dataset
          samplewise_center=False,  # set each sample mean to 0
          featurewise_std_normalization=False,  # divide inputs by std of the dataset
          samplewise_std_normalization=False,  # divide each input by its std
          zca_whitening=False,  # apply ZCA whitening
          rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
          width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
          height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
          horizontal_flip=True,  # randomly flip images
          vertical_flip=False)  # randomly flip images

      # Compute quantities required for featurewise normalization
      # (std, mean, and principal components if ZCA whitening is applied).
      datagen.fit(X_train)

      # Fit the model on the batches generated by datagen.flow().
      model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                          steps_per_epoch=X_train.shape[0] // batch_size,
                          validation_data=(X_test, Y_test),
                          epochs=epochs, verbose=1, max_q_size=1000,
                          callbacks=[lr_reducer, early_stopper, csv_logger])


### Train without augmentation 

1. Expected validation accuracy is around 40% with close to 100% training accurcy.
2. Don't forget to observe overfitting symptoms



In [6]:
Train(data_augmentation=False)

Not using data augmentation.


W0622 00:04:06.292895 140196683085696 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 4000 samples, validate on 1000 samples
Epoch 1/200
4000/4000 [==============================] - 11s 3ms/step - loss: 2.7331 - acc: 0.2860 - val_loss: 2.8066 - val_acc: 0.2980
Epoch 2/200
4000/4000 [==============================] - 4s 1ms/step - loss: 2.3146 - acc: 0.3985 - val_loss: 2.5168 - val_acc: 0.3390
Epoch 3/200
4000/4000 [==============================] - 4s 1ms/step - loss: 2.0972 - acc: 0.4645 - val_loss: 2.6300 - val_acc: 0.3220
Epoch 4/200
4000/4000 [==============================] - 4s 1ms/step - loss: 1.8367 - acc: 0.5655 - val_loss: 2.7193 - val_acc: 0.3150
Epoch 5/200
4000/4000 [==============================] - 4s 1ms/step - loss: 1.5696 - acc: 0.6510 - val_loss: 2.8232 - val_acc: 0.3210
Epoch 6/200
4000/4000 [==============================] - 4s 1ms/step - loss: 1.2792 - acc: 0.7507 - val_loss: 3.2000 - val_acc: 0.3380
Epoch 7/200
4000/4000 [==============================] - 4s 1ms/step - loss: 1.0691 - acc: 0.8317 - val_loss: 3.1813 - val_acc: 0.3430
Epoch 

### Train with image augmentation 

1. Expected validation accuracy is around 50% with training accuracy under 80%.
2. Overfitting is reduced with augmentation

---

### more data, better results


In [7]:
Train(data_augmentation=True)

Using real-time data augmentation.
Epoch 1/200
  3/125 [..............................] - ETA: 5s - loss: 2.0009 - acc: 0.6667

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=125, validation_data=(array([[[..., epochs=200, verbose=1, callbacks=[<keras.ca..., max_queue_size=1000)`


125/125 [==============================] - 5s 41ms/step - loss: 2.1083 - acc: 0.5533 - val_loss: 2.7808 - val_acc: 0.4030
Epoch 2/200
125/125 [==============================] - 5s 40ms/step - loss: 1.7500 - acc: 0.5915 - val_loss: 2.4465 - val_acc: 0.4040
Epoch 3/200
125/125 [==============================] - 5s 40ms/step - loss: 1.6783 - acc: 0.6068 - val_loss: 2.2862 - val_acc: 0.4210
Epoch 4/200
125/125 [==============================] - 5s 39ms/step - loss: 1.6238 - acc: 0.6322 - val_loss: 2.2268 - val_acc: 0.4310
Epoch 5/200
125/125 [==============================] - 5s 40ms/step - loss: 1.5804 - acc: 0.6422 - val_loss: 2.2231 - val_acc: 0.4540
Epoch 6/200
125/125 [==============================] - 5s 39ms/step - loss: 1.5723 - acc: 0.6420 - val_loss: 2.2024 - val_acc: 0.4350
Epoch 7/200
125/125 [==============================] - 5s 39ms/step - loss: 1.5261 - acc: 0.6555 - val_loss: 2.1805 - val_acc: 0.4630
Epoch 8/200
125/125 [==============================] - 5s 39ms/step - loss